In [36]:
import numpy as np
np.random.seed(12345)

def random_u64():
    return int(
        np.random.default_rng().integers(
            0, 2**64, dtype=np.uint64
        )
    )

def random_sign():
    return np.random.choice(np.array([1, -1], dtype=np.int8))

def random_limbs(max_size=3):
    size = np.random.randint(1, max_size+1)
    sign = random_sign()
    return (sign, [random_u64() for _ in range(size)])

def limbs_to_int(limbs):
    value = 0
    for limb in reversed(limbs):
        value = (value << 64) + limb
    return value

U64_MASK = 0xFFFFFFFFFFFFFFFF
def int_to_limbs(value: int) -> list[int]:
    if value < 0:
        raise ValueError("value must be non-negative")
    if value == 0:
        return [0]

    limbs = []
    while value:
        limbs.append(value & U64_MASK)
        value >>= 64
    return limbs        

In [37]:
for _ in range(10):
    print(random_limbs())

(-1, [16604713711210550425, 456097925126562877, 16287269308176174709])
(-1, [8757919866730572639, 13915614074873598326])
(-1, [12829473951434293559])
(1, [4466015102769624403, 5045267652375082677, 8786915615937031174])
(1, [5505322988719671108, 11644717669353196615])
(-1, [14133652682727641361, 12876230678777975174])
(-1, [10293649176498188491, 7742058478728126091, 6503160421724638655])
(-1, [7437270410710348310, 8083462288731850112, 340260485839294704])
(1, [13044380438678640401, 11869612818100881865, 3733987853663917774])
(-1, [13321859175596076699, 18075876023211539790, 17336197203310350896])


In [76]:
import operator

OPS = {
    "+": operator.add,
    "-": operator.sub,
    "*": operator.mul,
}

def sign(x):
    return (x > 0) - (x < 0)

def generate_use_cases(num=1):
    na = [random_limbs(max_size=10) for _ in range(num)]
    nb = [random_limbs(max_size=10) for _ in range(num)]
    cases = []

    for a, b in zip(na, nb):
        # a, b are assumed like: [sign, limbs]
        xa = int(a[0] * limbs_to_int(a[1]))
        xb = int(b[0] * limbs_to_int(b[1]))

        op_symbol = np.random.choice(list(OPS.keys()))
        xc = OPS[op_symbol](xa, xb)

        case = {
            "a_sign": int(sign(xa)),
            "a_limbs": [int(x) for x in a[1]],
            "b_sign": int(sign(xb)),
            "b_limbs": [int(x) for x in b[1]],
            "op": str(op_symbol),
            "c_sign": int(sign(xc)),
            "c_limbs": [int(x) for x in int_to_limbs(abs(xc))],
        }
        cases.append(case)

    return cases

In [77]:
cases = generate_use_cases(20)

In [78]:
cases

[{'a_sign': -1,
  'a_limbs': [16362869968789633489,
   12016630898417689170,
   11354423363985624876,
   16613182794197762079,
   17224842555786912957,
   3514554074612120239,
   6466733003993105510],
  'b_sign': 1,
  'b_limbs': [15669335706744843196,
   16221837992768815100,
   112888971429266063,
   3330961920011854077,
   11737275159301615628,
   17468860036043777382,
   9324588634849573180,
   772591364323205734,
   6850879030575372456],
  'op': '*',
  'c_sign': -1,
  'c_limbs': [4564331677691170940,
   12656241350059354586,
   10234741825547641214,
   7739064247594168092,
   2884080254348322918,
   2661040772751181587,
   2372813154030266091,
   1374100508444537413,
   7406838851622630940,
   12512321239371041387,
   1283956897236078749,
   13964806401871053156,
   12953536898352385824,
   4808048175928548964,
   5329289575777942297,
   2401659900324999277]},
 {'a_sign': 1,
  'a_limbs': [7804087670407921751,
   5572409966967505532,
   14504340286381066637,
   1462490832290290957,


In [79]:



def emit_rust_tests(
    cases,
    module_name: str = "generated_bignum_tests",
    test_fn_name: str = "bignum_use_cases",
) -> str:
    """
    Emit a self-contained Rust #[cfg(test)] module.

    IMPORTANT: It uses &'static [u64] slices in the const table (NO vec![] in const),
    then converts to Vec<u64> at runtime in mk().
    """

    def fmt_slice_u64(limbs) -> str:
        if not limbs:
            return "&[]"
        return "&[" + ", ".join(f"{int(x)}u64" for x in limbs) + "]"

    rows = []
    for c in cases:
        rows.append(
            "        Case { "
            f"a_sign: {int(c['a_sign'])}, a_limbs: {fmt_slice_u64(c['a_limbs'])}, "
            f"b_sign: {int(c['b_sign'])}, b_limbs: {fmt_slice_u64(c['b_limbs'])}, "
            f"op: '{c['op']}', "
            f"c_sign: {int(c['c_sign'])}, c_limbs: {fmt_slice_u64(c['c_limbs'])} "
            "},"
        )

    rust = f"""\
#[cfg(test)]
mod {module_name} {{
    use super::*;
    use MSgn::*;

    #[derive(Clone, Debug)]
    struct Case {{
        a_sign: i8,
        a_limbs: &'static [u64],
        b_sign: i8,
        b_limbs: &'static [u64],
        op: char,
        c_sign: i8,
        c_limbs: &'static [u64],
    }}

    fn sgn_from_i8(x: i8) -> MSgn {{
        match x {{
            -1 => MNeg,
             0 => MZero,
             1 => MPos,
             _ => panic!("invalid sign: {{}}", x),
        }}
    }}

    // Normalize to canonical representation:
    // - trim high zero limbs (little-endian => trim from the end)
    // - if limbs become empty => sign must be zero
    fn normalize(mut x: MarInt) -> MarInt {{
        while x.limbs.last().copied() == Some(0) {{
            x.limbs.pop();
        }}
        if x.limbs.is_empty() {{
            x.sign = MZero;
        }}
        x
    }}

    fn mk(sign: i8, limbs: &'static [u64]) -> MarInt {{
        normalize(MarInt {{
            sign: sgn_from_i8(sign),
            limbs: limbs.to_vec(),
        }})
    }}

    const CASES: &[Case] = &[
{chr(10).join(rows)}
    ];

    #[test]
    fn {test_fn_name}() {{
        for (i, tc) in CASES.iter().enumerate() {{
            let a = mk(tc.a_sign, tc.a_limbs);
            let b = mk(tc.b_sign, tc.b_limbs);
            let expected = mk(tc.c_sign, tc.c_limbs);

            // If your ops are implemented for &MarInt instead of owned MarInt,
            // change these lines to: &a + &b, &a - &b, &a * &b
            let got = match tc.op {{
                '+' => normalize(a.clone() + b.clone()),
                '-' => normalize(a.clone() - b.clone()),
                '*' => normalize(a.clone() * b.clone()),
                _ => panic!("unknown op: {{:?}}", tc.op),
            }};

            assert_eq!(
                got.sign as i8, expected.sign as i8,
                "case {{}} sign mismatch: op={{}} a={{:?}} b={{:?}} got={{:?}} expected={{:?}}",
                i, tc.op, a, b, got, expected
            );
            assert_eq!(
                got.limbs, expected.limbs,
                "case {{}} limbs mismatch: op={{}} a={{:?}} b={{:?}} got={{:?}} expected={{:?}}",
                i, tc.op, a, b, got, expected
            );
        }}
    }}
}}
"""
    return rust

In [80]:
print(emit_rust_tests(cases, module_name="generated_marint_tests", test_fn_name="marint_random_cases"))

#[cfg(test)]
mod generated_marint_tests {
    use super::*;
    use MSgn::*;

    #[derive(Clone, Debug)]
    struct Case {
        a_sign: i8,
        a_limbs: &'static [u64],
        b_sign: i8,
        b_limbs: &'static [u64],
        op: char,
        c_sign: i8,
        c_limbs: &'static [u64],
    }

    fn sgn_from_i8(x: i8) -> MSgn {
        match x {
            -1 => MNeg,
             0 => MZero,
             1 => MPos,
             _ => panic!("invalid sign: {}", x),
        }
    }

    // Normalize to canonical representation:
    // - trim high zero limbs (little-endian => trim from the end)
    // - if limbs become empty => sign must be zero
    fn normalize(mut x: MarInt) -> MarInt {
        while x.limbs.last().copied() == Some(0) {
            x.limbs.pop();
        }
        if x.limbs.is_empty() {
            x.sign = MZero;
        }
        x
    }

    fn mk(sign: i8, limbs: &'static [u64]) -> MarInt {
        normalize(MarInt {
            sign: sgn_from_i8(sig

In [55]:
14164991695968020480*-327291290990679900968828123809255349248

-4636078419045633792231490460902731251679579286981616599040

In [ ]:
{'a_sign': -1,
  'a_limbs': [14782120271727931364],
  'b_sign': 1,
  'b_limbs': [11362751999683767719],
  'op': '+',
  'c_sign': -1,
  'c_limbs': [3419368272044163072]}

In [71]:
14782120271727931364-11362751999683767719

3419368272044163645

In [74]:
(-limbs_to_int([14782120271727931364]))+(limbs_to_int([11362751999683767719]))

-3419368272044163645

In [75]:
[int(x) for x in int_to_limbs(abs(-3419368272044163645))]

[3419368272044163645]